In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
import xgboost as xgb

pd.set_option("display.max_columns", None)
%matplotlib inline

In [2]:
car_df = pd.read_csv("train-data.csv")
car_df.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [3]:
# checking the information about data
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         6019 non-null   int64  
 1   Name               6019 non-null   object 
 2   Location           6019 non-null   object 
 3   Year               6019 non-null   int64  
 4   Kilometers_Driven  6019 non-null   int64  
 5   Fuel_Type          6019 non-null   object 
 6   Transmission       6019 non-null   object 
 7   Owner_Type         6019 non-null   object 
 8   Mileage            6017 non-null   object 
 9   Engine             5983 non-null   object 
 10  Power              5983 non-null   object 
 11  Seats              5977 non-null   float64
 12  New_Price          824 non-null    object 
 13  Price              6019 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 658.5+ KB


It seems that there are updated price with old price so we will be replacing the old price with updated price if given 
else we will keep the same price.

In [4]:
## dropping the columns unnamed by capturing all the columns with names starting Unnamed
unnammed_cols = car_df.columns[car_df.columns.str.contains("Unnamed")].to_list()
car_df.drop(columns=unnammed_cols, axis=1, inplace=True)
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  New_Price          824 non-null    object 
 12  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 611.4+ KB


In [5]:
def update_price(df):
    """
    Check for the all index where new price is not null or nan 
    then price gets updated with new price.
    """
    indexes = df[pd.notnull(df.New_Price)].index
    for index in indexes:
        df.loc[index, "Price"] = df.loc[index, "New_Price"]
            
update_price(car_df)
car_df.head(10)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.5
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,8.61 Lakh
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
5,Hyundai EON LPG Era Plus Option,Hyderabad,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,NaN,2.35
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.5
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,21 Lakh
8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,NaN,5.2
9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,NaN,1.95


In [6]:
# Checking that if Price has been updated or not
car_df[pd.notnull(car_df.New_Price)]

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,8.61 Lakh
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,21 Lakh
10,Maruti Ciaz Zeta,Kochi,2018,25692,Petrol,Manual,First,21.56 kmpl,1462 CC,103.25 bhp,5.0,10.65 Lakh,10.65 Lakh
15,Mitsubishi Pajero Sport 4X4,Delhi,2014,110000,Diesel,Manual,First,13.5 kmpl,2477 CC,175.56 bhp,7.0,32.01 Lakh,32.01 Lakh
20,BMW 3 Series 320d,Kochi,2014,32982,Diesel,Automatic,First,22.69 kmpl,1995 CC,190 bhp,5.0,47.87 Lakh,47.87 Lakh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999,Tata Bolt Revotron XT,Chennai,2016,10000,Petrol,Manual,First,17.57 kmpl,1193 CC,88.7 bhp,5.0,7.77 Lakh,7.77 Lakh
6002,Volkswagen Vento 1.6 Highline,Mumbai,2011,38000,Petrol,Manual,First,16.09 kmpl,1598 CC,103.5 bhp,5.0,11.91 Lakh,11.91 Lakh
6005,Maruti Vitara Brezza VDi,Pune,2016,37208,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,9.93 Lakh,9.93 Lakh
6010,Honda Brio 1.2 VX MT,Delhi,2013,33746,Petrol,Manual,First,18.5 kmpl,1198 CC,86.8 bhp,5.0,6.63 Lakh,6.63 Lakh


In [7]:
# Checking for null values in the pandas df.
car_df.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

#### we will drop the new price columns and then drop the records where Engine, Power, Seats, and Mileage are null as they are main indicator for Price determination.

In [8]:
car_df.drop(columns=["New_Price", "Location"], axis=1, inplace=True)
car_df.head(5)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.5
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.0
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


In [9]:
car_df.dropna(axis=0, inplace=True)
car_df.isna().sum()

Name                 0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [10]:
car_df.reset_index(drop=True, inplace=True)
car_df.tail(10)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
5965,Toyota Camry Hybrid,2015,33500,Petrol,Automatic,First,19.16 kmpl,2494 CC,158.2 bhp,5.0,19.75
5966,Honda Brio 1.2 VX MT,2013,33746,Petrol,Manual,First,18.5 kmpl,1198 CC,86.8 bhp,5.0,6.63 Lakh
5967,Skoda Superb 3.6 V6 FSI,2009,53000,Petrol,Automatic,First,0.0 kmpl,3597 CC,262.6 bhp,5.0,4.75
5968,Toyota Innova 2.5 V Diesel 7-seater,2011,45004,Diesel,Manual,First,12.8 kmpl,2494 CC,102 bhp,7.0,9.48
5969,Honda Amaze VX i-DTEC,2015,70602,Diesel,Manual,First,25.8 kmpl,1498 CC,98.6 bhp,5.0,4.83
5970,Maruti Swift VDI,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,7.88 Lakh
5971,Hyundai Xcent 1.1 CRDi S,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,4.0
5972,Mahindra Xylo D4 BSIV,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,2.9
5973,Maruti Wagon R VXI,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,2.65
5974,Chevrolet Beat Diesel,2011,47000,Diesel,Manual,First,25.44 kmpl,936 CC,57.6 bhp,5.0,2.5


In [11]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5975 entries, 0 to 5974
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               5975 non-null   object 
 1   Year               5975 non-null   int64  
 2   Kilometers_Driven  5975 non-null   int64  
 3   Fuel_Type          5975 non-null   object 
 4   Transmission       5975 non-null   object 
 5   Owner_Type         5975 non-null   object 
 6   Mileage            5975 non-null   object 
 7   Engine             5975 non-null   object 
 8   Power              5975 non-null   object 
 9   Seats              5975 non-null   float64
 10  Price              5975 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 513.6+ KB


In [12]:
# Converting the fuel type with columns numerical values
print(car_df.Fuel_Type.unique())
# lets not use get_dummies method here but lets do it using a simple lambad function
car_df["Fuel_Type_Petrol"] = car_df.Fuel_Type.map(lambda x: 1 if x=="Petrol" else 0)
car_df["Fuel_Type_Diesel"] = car_df.Fuel_Type.map(lambda x: 1 if x=="Diesel" else 0)
car_df["Fuel_Type_CNG"] = car_df.Fuel_Type.map(lambda x: 1 if x=="CNG" else 0)
# we dont need to add LPG as it can be represneted with these three columns [0,0,0] can be intrepreted as 1 for LPG, 
# implying the car runs on LPG.

['CNG' 'Diesel' 'Petrol' 'LPG']


In [13]:
car_df.head(10)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75,0,0,1
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.5,0,1,0
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,1,0,0
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.0,0,1,0
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74,0,1,0
5,Hyundai EON LPG Era Plus Option,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,2.35,0,0,0
6,Nissan Micra Diesel XV,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,3.5,0,1,0
7,Toyota Innova Crysta 2.8 GX AT 8S,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,0,1,0
8,Volkswagen Vento Diesel Comfortline,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,5.2,0,1,0
9,Tata Indica Vista Quadrajet LS,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,1.95,0,1,0


In [14]:
# Converting the Transmission with numerical columns values (one hot encoding)
print(car_df.Transmission.unique())
car_df["Automatic_Transmission"] = car_df.Transmission.map(lambda x: 1 if x == "Automatic" else 0)
# we don't need to add new columns for manual transmission because in our dataset.
# we have only two types and if Automatic_Transmission is 0 then we will consider it as manual.

['Manual' 'Automatic']


In [15]:
car_df.head(10)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75,0,0,1,0
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.5,0,1,0,0
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,1,0,0,0
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.0,0,1,0,0
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74,0,1,0,1
5,Hyundai EON LPG Era Plus Option,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,2.35,0,0,0,0
6,Nissan Micra Diesel XV,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,3.5,0,1,0,0
7,Toyota Innova Crysta 2.8 GX AT 8S,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,0,1,0,1
8,Volkswagen Vento Diesel Comfortline,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,5.2,0,1,0,0
9,Tata Indica Vista Quadrajet LS,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,1.95,0,1,0,0


In [16]:
# Converting the Owner_Type with numerical columns values (one hot encoding)
print(car_df.Owner_Type.unique())
car_df["Owner_Type_First"] = car_df.Owner_Type.map(lambda x: 1 if x == "First" else 0)
car_df["Owner_Type_Second"] = car_df.Owner_Type.map({"First": 0, "Second": 1, "Third": 0, "Fourth & Above": 0})
car_df["Owner_Type_Third"] = car_df.Owner_Type.map({"First": 0, "Second": 0, "Third": 1, "Fourth & Above": 0})
# we don't need to add new columns for Fourth & Above ownership because in our dataset.
# we have only four types and if all three ownership is [0, 0, 0] then we will consider it as Fourth & Above.

['First' 'Second' 'Fourth & Above' 'Third']


In [17]:
car_df[car_df.Owner_Type_Third ==1]

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third
54,Hyundai Grand i10 Sportz,2015,54000,Petrol,Manual,Third,18.9 kmpl,1197 CC,82 bhp,5.0,3.6,1,0,0,0,0,0,1
77,Toyota Innova 2.0 G1,2006,230000,Petrol,Manual,Third,12.4 kmpl,1998 CC,132 bhp,8.0,4.5,1,0,0,0,0,0,1
143,Hyundai Santro Xing XO eRLX Euro II,2008,80759,Petrol,Manual,Third,17.0 kmpl,1086 CC,null bhp,5.0,1.67,1,0,0,0,0,0,1
242,Hyundai Santro Xing XO eRLX Euro II,2007,72000,Petrol,Manual,Third,17.0 kmpl,1086 CC,null bhp,5.0,0.95,1,0,0,0,0,0,1
254,Maruti Omni MPI CARGO BSIII W/ IMMOBILISER,2011,35000,Petrol,Manual,Third,19.7 kmpl,796 CC,35 bhp,5.0,1.85,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5673,Maruti Zen LX,1998,95150,Petrol,Manual,Third,17.3 kmpl,993 CC,60 bhp,5.0,0.53,1,0,0,0,0,0,1
5738,Lamborghini Gallardo Coupe,2011,6500,Petrol,Automatic,Third,6.4 kmpl,5204 CC,560 bhp,2.0,120.0,1,0,0,1,0,0,1
5898,Ford Endeavour 2.5L 4X2 MT,2011,119000,Diesel,Manual,Third,13.1 kmpl,2499 CC,141 bhp,7.0,6.5,0,1,0,0,0,0,1
5946,Tata New Safari DICOR 2.2 VX 4x2,2008,86000,Diesel,Manual,Third,13.93 kmpl,2179 CC,138 bhp,7.0,0.6,0,1,0,0,0,0,1


In [18]:
# Convert the mileage into the numerical values but we have the unit as kmpl and km/kg. 
# There is no diect conversion rule for the gas into liters as volume depends on pressure and temperature.
# So lets remove the kmpl and km/kg from the milegae and add new column fuel_type {liquid: 1 and gas: 0}, 
# CNG and LPG are gas rest are liters.
car_df["Fuel_Type_Liqiuid"] = car_df.Mileage.map(lambda x: 0 if "kg" in x else 1)
car_df["Mileage"] = car_df.Mileage.map(lambda x: float(x.split()[0]))

In [19]:
car_df.head(10)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.60,998 CC,58.16 bhp,5.0,1.75,0,0,1,0,1,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67,1582 CC,126.2 bhp,5.0,12.5,0,1,0,0,1,0,0,1
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.20,1199 CC,88.7 bhp,5.0,8.61 Lakh,1,0,0,0,1,0,0,1
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77,1248 CC,88.76 bhp,7.0,6.0,0,1,0,0,1,0,0,1
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.20,1968 CC,140.8 bhp,5.0,17.74,0,1,0,1,0,1,0,1
5,Hyundai EON LPG Era Plus Option,2012,75000,LPG,Manual,First,21.10,814 CC,55.2 bhp,5.0,2.35,0,0,0,0,1,0,0,0
6,Nissan Micra Diesel XV,2013,86999,Diesel,Manual,First,23.08,1461 CC,63.1 bhp,5.0,3.5,0,1,0,0,1,0,0,1
7,Toyota Innova Crysta 2.8 GX AT 8S,2016,36000,Diesel,Automatic,First,11.36,2755 CC,171.5 bhp,8.0,21 Lakh,0,1,0,1,1,0,0,1
8,Volkswagen Vento Diesel Comfortline,2013,64430,Diesel,Manual,First,20.54,1598 CC,103.6 bhp,5.0,5.2,0,1,0,0,1,0,0,1
9,Tata Indica Vista Quadrajet LS,2012,65932,Diesel,Manual,Second,22.30,1248 CC,74 bhp,5.0,1.95,0,1,0,0,0,1,0,1


In [20]:
# extracting numerical values from engine
car_df["Engine_in_CC"] = car_df.Engine.map(lambda x: int(x.split()[0]))
car_df.head(10)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.60,998 CC,58.16 bhp,5.0,1.75,0,0,1,0,1,0,0,0,998
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67,1582 CC,126.2 bhp,5.0,12.5,0,1,0,0,1,0,0,1,1582
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.20,1199 CC,88.7 bhp,5.0,8.61 Lakh,1,0,0,0,1,0,0,1,1199
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77,1248 CC,88.76 bhp,7.0,6.0,0,1,0,0,1,0,0,1,1248
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.20,1968 CC,140.8 bhp,5.0,17.74,0,1,0,1,0,1,0,1,1968
5,Hyundai EON LPG Era Plus Option,2012,75000,LPG,Manual,First,21.10,814 CC,55.2 bhp,5.0,2.35,0,0,0,0,1,0,0,0,814
6,Nissan Micra Diesel XV,2013,86999,Diesel,Manual,First,23.08,1461 CC,63.1 bhp,5.0,3.5,0,1,0,0,1,0,0,1,1461
7,Toyota Innova Crysta 2.8 GX AT 8S,2016,36000,Diesel,Automatic,First,11.36,2755 CC,171.5 bhp,8.0,21 Lakh,0,1,0,1,1,0,0,1,2755
8,Volkswagen Vento Diesel Comfortline,2013,64430,Diesel,Manual,First,20.54,1598 CC,103.6 bhp,5.0,5.2,0,1,0,0,1,0,0,1,1598
9,Tata Indica Vista Quadrajet LS,2012,65932,Diesel,Manual,Second,22.30,1248 CC,74 bhp,5.0,1.95,0,1,0,0,0,1,0,1,1248


In [21]:
# extracting numerical values from Power
car_df["Power_in_bph"] = car_df.Power.map(lambda x:  x.split()[0])
car_df.head(5)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.60,998 CC,58.16 bhp,5.0,1.75,0,0,1,0,1,0,0,0,998,58.16
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67,1582 CC,126.2 bhp,5.0,12.5,0,1,0,0,1,0,0,1,1582,126.2
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.20,1199 CC,88.7 bhp,5.0,8.61 Lakh,1,0,0,0,1,0,0,1,1199,88.7
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77,1248 CC,88.76 bhp,7.0,6.0,0,1,0,0,1,0,0,1,1248,88.76
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.20,1968 CC,140.8 bhp,5.0,17.74,0,1,0,1,0,1,0,1,1968,140.8


In [22]:
car_df[car_df.Power_in_bph == "null"]

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph
76,Ford Fiesta 1.4 SXi TDCi,2008,111111,Diesel,Manual,First,17.80,1399 CC,null bhp,5.0,2.0,0,1,0,0,1,0,0,1,1399,null
79,Hyundai Santro Xing XL,2005,87591,Petrol,Manual,First,0.00,1086 CC,null bhp,5.0,1.3,1,0,0,0,1,0,0,1,1086,null
89,Hyundai Santro Xing XO,2007,73745,Petrol,Manual,First,17.00,1086 CC,null bhp,5.0,2.1,1,0,0,0,1,0,0,1,1086,null
120,Hyundai Santro Xing XL eRLX Euro III,2005,102000,Petrol,Manual,Second,17.00,1086 CC,null bhp,5.0,0.85,1,0,0,0,0,1,0,1,1086,null
143,Hyundai Santro Xing XO eRLX Euro II,2008,80759,Petrol,Manual,Third,17.00,1086 CC,null bhp,5.0,1.67,1,0,0,0,0,0,1,1,1086,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5818,Hyundai Santro Xing XO,2007,79000,Petrol,Manual,First,17.00,1086 CC,null bhp,5.0,1.85,1,0,0,0,1,0,0,1,1086,null
5830,Hyundai Santro Xing XO eRLX Euro II,2006,47200,Petrol,Manual,Second,17.00,1086 CC,null bhp,5.0,1.2,1,0,0,0,0,1,0,1,1086,null
5881,Skoda Laura Classic 1.8 TSI,2010,85000,Petrol,Manual,First,17.50,1798 CC,null bhp,5.0,2.85,1,0,0,0,1,0,0,1,1798,null
5899,Mahindra Jeep MM 540 DP,2002,75000,Diesel,Manual,First,0.00,2112 CC,null bhp,6.0,1.7,0,1,0,0,1,0,0,1,2112,null


In [23]:
# There are some null bph in the Power 
indexes = car_df[car_df.Power_in_bph == "null"].index
indexes=indexes.to_list()

In [24]:
new_car_df = car_df.drop(indexes)
new_car_df[new_car_df.Power_in_bph =="null"]
new_car_df.reset_index(drop=True, inplace=True)

In [25]:
new_car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5872 entries, 0 to 5871
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    5872 non-null   object 
 1   Year                    5872 non-null   int64  
 2   Kilometers_Driven       5872 non-null   int64  
 3   Fuel_Type               5872 non-null   object 
 4   Transmission            5872 non-null   object 
 5   Owner_Type              5872 non-null   object 
 6   Mileage                 5872 non-null   float64
 7   Engine                  5872 non-null   object 
 8   Power                   5872 non-null   object 
 9   Seats                   5872 non-null   float64
 10  Price                   5872 non-null   object 
 11  Fuel_Type_Petrol        5872 non-null   int64  
 12  Fuel_Type_Diesel        5872 non-null   int64  
 13  Fuel_Type_CNG           5872 non-null   int64  
 14  Automatic_Transmission  5872 non-null   

In [26]:
# Convert the datatype of power_in_bph to float and Seats to int
new_car_df.Power_in_bph = new_car_df.Power_in_bph.astype("float64")
new_car_df.Seats = new_car_df.Seats.astype("int64")
new_car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5872 entries, 0 to 5871
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    5872 non-null   object 
 1   Year                    5872 non-null   int64  
 2   Kilometers_Driven       5872 non-null   int64  
 3   Fuel_Type               5872 non-null   object 
 4   Transmission            5872 non-null   object 
 5   Owner_Type              5872 non-null   object 
 6   Mileage                 5872 non-null   float64
 7   Engine                  5872 non-null   object 
 8   Power                   5872 non-null   object 
 9   Seats                   5872 non-null   int64  
 10  Price                   5872 non-null   object 
 11  Fuel_Type_Petrol        5872 non-null   int64  
 12  Fuel_Type_Diesel        5872 non-null   int64  
 13  Fuel_Type_CNG           5872 non-null   int64  
 14  Automatic_Transmission  5872 non-null   

In [27]:
new_car_df["Price_in_inr_lakh"] = new_car_df.Price.map(lambda x: float(str(x).split()[0]))
new_car_df.head(5)

,Name,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_inr_lakh
0,Maruti Wagon R LXI CNG,2010,72000,CNG,Manual,First,26.60,998 CC,58.16 bhp,5,1.75,0,0,1,0,1,0,0,0,998,58.16,1.75
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,Diesel,Manual,First,19.67,1582 CC,126.2 bhp,5,12.5,0,1,0,0,1,0,0,1,1582,126.20,12.50
2,Honda Jazz V,2011,46000,Petrol,Manual,First,18.20,1199 CC,88.7 bhp,5,8.61 Lakh,1,0,0,0,1,0,0,1,1199,88.70,8.61
3,Maruti Ertiga VDI,2012,87000,Diesel,Manual,First,20.77,1248 CC,88.76 bhp,7,6.0,0,1,0,0,1,0,0,1,1248,88.76,6.00
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,Diesel,Automatic,Second,15.20,1968 CC,140.8 bhp,5,17.74,0,1,0,1,0,1,0,1,1968,140.80,17.74


In [28]:
new_car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5872 entries, 0 to 5871
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    5872 non-null   object 
 1   Year                    5872 non-null   int64  
 2   Kilometers_Driven       5872 non-null   int64  
 3   Fuel_Type               5872 non-null   object 
 4   Transmission            5872 non-null   object 
 5   Owner_Type              5872 non-null   object 
 6   Mileage                 5872 non-null   float64
 7   Engine                  5872 non-null   object 
 8   Power                   5872 non-null   object 
 9   Seats                   5872 non-null   int64  
 10  Price                   5872 non-null   object 
 11  Fuel_Type_Petrol        5872 non-null   int64  
 12  Fuel_Type_Diesel        5872 non-null   int64  
 13  Fuel_Type_CNG           5872 non-null   int64  
 14  Automatic_Transmission  5872 non-null   

In [29]:
# Lets drop the columns Fuel_Type, Transmission, Owner_Type, Engine, Power, Price
new_car_df.drop(columns=["Fuel_Type", "Transmission", "Owner_Type", "Engine", "Power", "Price"], axis=1, inplace=True)
new_car_df.head(5)

,Name,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_inr_lakh
0,Maruti Wagon R LXI CNG,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,1.75
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,12.50
2,Honda Jazz V,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,8.61
3,Maruti Ertiga VDI,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,6.00
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,17.74


In [30]:
# Most of the car are imported into nepali market from indian market so we will convert the price in INR to NPR.
# Total import duties for car above 1000 cc is 80% customs duty and up to 80% excise duty, 13% VAT, and 7% road construction fee.
# Total import duties for car below 1000 cc is 80% customs duty and up to 40% excise duty, 13% VAT, and 7% road construction fee.
# Ofcourse there are some factors like nepali public inclination towards brand, transportation fee and other.
# Lets ignore these factors for now.
# 1000 cc above: 180 percent
# 1000 cc below: 140 percent
# 1 inr = 1.6 npr

def converting_to_nepali_market_value(df):
    num_rows = len(df)
    npr_price = []
    for i in range(num_rows):
        if df.Engine_in_CC[i]>1000:
            npr_price.append(round((df["Price_in_inr_lakh"][i]*1.6) * 2.8, 2))
        elif df.Engine_in_CC[i]<1000:
            npr_price.append(round((df["Price_in_inr_lakh"][i]*1.6) * 2.4, 2))
    df["Price_in_npr"] = npr_price

converting_to_nepali_market_value(new_car_df)
new_car_df.head(5)

,Name,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_inr_lakh,Price_in_npr
0,Maruti Wagon R LXI CNG,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,1.75,6.72
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,12.50,56.00
2,Honda Jazz V,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,8.61,38.57
3,Maruti Ertiga VDI,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,6.00,26.88
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,17.74,79.48


In [31]:
new_car_df.Name[0].split()[0]

'Maruti'

In [32]:
new_car_df["Brand_Name"] = new_car_df.Name.map(lambda x: x.split()[0])
new_car_df.head(10)

,Name,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_inr_lakh,Price_in_npr,Brand_Name
0,Maruti Wagon R LXI CNG,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,1.75,6.72,Maruti
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,12.50,56.00,Hyundai
2,Honda Jazz V,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,8.61,38.57,Honda
3,Maruti Ertiga VDI,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,6.00,26.88,Maruti
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,17.74,79.48,Audi
5,Hyundai EON LPG Era Plus Option,2012,75000,21.10,5,0,0,0,0,1,0,0,0,814,55.20,2.35,9.02,Hyundai
6,Nissan Micra Diesel XV,2013,86999,23.08,5,0,1,0,0,1,0,0,1,1461,63.10,3.50,15.68,Nissan
7,Toyota Innova Crysta 2.8 GX AT 8S,2016,36000,11.36,8,0,1,0,1,1,0,0,1,2755,171.50,21.00,94.08,Toyota
8,Volkswagen Vento Diesel Comfortline,2013,64430,20.54,5,0,1,0,0,1,0,0,1,1598,103.60,5.20,23.30,Volkswagen
9,Tata Indica Vista Quadrajet LS,2012,65932,22.30,5,0,1,0,0,0,1,0,1,1248,74.00,1.95,8.74,Tata


In [33]:
new_car_df.drop(columns=["Name", "Price_in_inr_lakh"], axis=1, inplace=True)
new_car_df.head(5)

,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_npr,Brand_Name
0,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,6.72,Maruti
1,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,56.00,Hyundai
2,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,38.57,Honda
3,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,26.88,Maruti
4,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,79.48,Audi


In [34]:
new_car_df.Brand_Name.value_counts()

Maruti           1175
Hyundai          1058
Honda             600
Toyota            394
Mercedes-Benz     316
Volkswagen        314
Ford              294
Mahindra          268
BMW               262
Audi              235
Tata              183
Skoda             172
Renault           145
Chevrolet         120
Nissan             89
Land               57
Jaguar             40
Mitsubishi         27
Mini               26
Fiat               23
Volvo              21
Porsche            16
Jeep               15
Datsun             13
Force               3
ISUZU               2
Ambassador          1
Isuzu               1
Bentley             1
Lamborghini         1
Name: Brand_Name, dtype: int64

In [35]:
indexes = new_car_df[new_car_df.Brand_Name == "Force"].index.values
indexes = indexes.tolist()

In [36]:
for index in new_car_df[new_car_df.Brand_Name == "ISUZU"].index.values:
    indexes.append(index)
indexes

[4870, 5052, 5075, 3530, 5833]

In [37]:
for index in new_car_df[new_car_df.Brand_Name == "Ambassador"].index.values:
    indexes.append(index)

In [38]:
for index in new_car_df[new_car_df.Brand_Name == "Isuzu"].index.values:
    indexes.append(index)
indexes

[4870, 5052, 5075, 3530, 5833, 1188, 2820]

In [39]:
for index in new_car_df[new_car_df.Brand_Name == "Bentley"].index.values:
    indexes.append(index)
indexes

[4870, 5052, 5075, 3530, 5833, 1188, 2820, 5385]

In [40]:
for index in new_car_df[new_car_df.Brand_Name == "Lamborghini"].index.values:
    indexes.append(index)
indexes

[4870, 5052, 5075, 3530, 5833, 1188, 2820, 5385, 5640]

In [41]:
new_car_df.drop(index=indexes, axis=0, inplace=True)
new_car_df[new_car_df.Brand_Name == "Lamborghini"]

,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_npr,Brand_Name


In [42]:
def brand_name(brand):
    if brand == "Maruti":
        return "non luxury"
    elif brand == "Hyundai":
        return "non luxury"
    elif brand == "Toyota":
        return "non luxury"
    elif brand == "Mahindra":
        return "non luxury"
    elif brand == "Tata":
        return "non luxury"
    elif brand == "Ambassador":
        return "non luxury"
    elif brand == "Volkswagen":
        return "non luxury"
    elif brand == "Datsun":
        return "non luxury"
    else:
        return "luxury"
new_car_df['Brand_Type'] = new_car_df.Brand_Name.apply(brand_name)
new_car_df.head(5)

,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_npr,Brand_Name,Brand_Type
0,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,6.72,Maruti,non luxury
1,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,56.00,Hyundai,non luxury
2,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,38.57,Honda,luxury
3,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,26.88,Maruti,non luxury
4,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,79.48,Audi,luxury


In [43]:
new_car_df.Brand_Type.unique()

array(['non luxury', 'luxury'], dtype=object)

In [44]:
new_car_df["Luxury_Brand"] = new_car_df.Brand_Type.map(lambda x: 1 if x=="luxury" else 0)
new_car_df.head(5)

,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_npr,Brand_Name,Brand_Type,Luxury_Brand
0,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,6.72,Maruti,non luxury,0
1,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,56.00,Hyundai,non luxury,0
2,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,38.57,Honda,luxury,1
3,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,26.88,Maruti,non luxury,0
4,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,79.48,Audi,luxury,1


In [45]:
final_df = new_car_df.drop(columns=["Brand_Name", "Brand_Type"], axis=1)
final_df.head(5)

,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Price_in_npr,Luxury_Brand
0,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,6.72,0
1,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,56.00,0
2,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,38.57,1
3,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,26.88,0
4,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,79.48,1


In [46]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5863 entries, 0 to 5871
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Year                    5863 non-null   int64  
 1   Kilometers_Driven       5863 non-null   int64  
 2   Mileage                 5863 non-null   float64
 3   Seats                   5863 non-null   int64  
 4   Fuel_Type_Petrol        5863 non-null   int64  
 5   Fuel_Type_Diesel        5863 non-null   int64  
 6   Fuel_Type_CNG           5863 non-null   int64  
 7   Automatic_Transmission  5863 non-null   int64  
 8   Owner_Type_First        5863 non-null   int64  
 9   Owner_Type_Second       5863 non-null   int64  
 10  Owner_Type_Third        5863 non-null   int64  
 11  Fuel_Type_Liqiuid       5863 non-null   int64  
 12  Engine_in_CC            5863 non-null   int64  
 13  Power_in_bph            5863 non-null   float64
 14  Price_in_npr            5863 non-null   

In [47]:
y = final_df["Price_in_npr"]
X = final_df.drop(columns=["Price_in_npr"], axis=0)
X.head(5)

,Year,Kilometers_Driven,Mileage,Seats,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Automatic_Transmission,Owner_Type_First,Owner_Type_Second,Owner_Type_Third,Fuel_Type_Liqiuid,Engine_in_CC,Power_in_bph,Luxury_Brand
0,2010,72000,26.60,5,0,0,1,0,1,0,0,0,998,58.16,0
1,2015,41000,19.67,5,0,1,0,0,1,0,0,1,1582,126.20,0
2,2011,46000,18.20,5,1,0,0,0,1,0,0,1,1199,88.70,1
3,2012,87000,20.77,7,0,1,0,0,1,0,0,1,1248,88.76,0
4,2013,40670,15.20,5,0,1,0,1,0,1,0,1,1968,140.80,1


In [48]:
# train test split 80:20
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=50)
X_train.shape, y_train.shape

((4690, 15), (4690,))

In [49]:
# applying the scaling
sc = StandardScaler()
X_train_sc =sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [50]:
rfg = RandomForestRegressor()
rfg.fit(X_train_sc, y_train)
y_pred = rfg.predict(X_test_sc)

In [51]:
from sklearn.metrics import r2_score, mean_absolute_error

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

9.742945433361752
0.849949305654148


In [52]:
xgb_reg = xgb.XGBRFRegressor()
xgb_reg.fit(X_train_sc, y_train)
y_pred = xgb_reg.predict(X_test_sc)

In [53]:
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

12.173934606469196
0.8166973184171556


In [54]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train_sc, y_train)
y_pred = gbr.predict(X_test_sc)

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

11.454860294212752
0.8391300129988546


In [55]:
ada_rg = AdaBoostRegressor()
ada_rg.fit(X_train_sc, y_train)
y_pred = ada_rg.predict(X_test_sc)

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

23.388268063393287
0.6651128448676493
